# Red hat customer value predction

# Importing data and preprocessing

In [131]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack
import xgboost as xgb

ImportError: No module named 'xgboost'

In [110]:
#defining function
def clean_data(data_set):
    clean_feature = ['year_x','month_x','day_x','isweekend_x','year_y','month_y','day_y','isweekend_y','outcome','char_38_y']
    colname = data_set.columns
    for i in colname:
        if i not in clean_feature:
            if(data_set[i].dtype == 'object'):
                data_set[i] = data_set[i].fillna('type 0')
                data_set[i] = data_set[i].apply(lambda x : x.split(' ')[1]).astype(np.int32)
            elif(data_set[i].dtype == 'bool'):
                data_set[i] = data_set[i].astype(np.int8)
    return data_set

In [111]:
people = pd.read_csv('./input/people.csv',dtype={'people_id':np.str,'char_38':np.int32},
                    parse_dates=['date'])
act_train = pd.read_csv('./input/act_train.csv',dtype={'people_id':np.str,'activity_id':np.str,'outcome':np.int8},
                        parse_dates=['date'])
act_test = pd.read_csv('./input/act_test.csv',dtype={'people_id':np.str,'activity_id':np.str},
                        parse_dates=['date'])
#Taking care of the date 
people['year'] = people['date'].dt.year
people['month'] = people['date'].dt.month
people['day'] = people['date'].dt.day
people['isweekend'] = (people['date'].dt.weekday>=5).astype(np.int8)
people = people.drop('date',axis=1)

act_train['year'] = act_train['date'].dt.year
act_train['month'] = act_train['date'].dt.month
act_train['day'] = act_train['date'].dt.day
act_train['isweekend'] = (act_train['date'].dt.weekday>=5).astype(np.int8)
act_train = act_train.drop('date',axis=1)

act_test['year'] = act_test['date'].dt.year
act_test['month'] = act_test['date'].dt.month
act_test['day'] = act_test['date'].dt.day
act_test['isweekend'] = (act_test['date'].dt.weekday>=5).astype(np.int8)
act_test = act_test.drop('date',axis=1)

train = act_train.merge(people, on='people_id', how='left', left_index=True)
test  = act_test.merge(people, on='people_id', how='left', left_index=True)

train = train.drop(['people_id','activity_id'],axis=1)
test = test.drop(['people_id','activity_id'],axis=1)
clean_data(train)
clean_data(test)

del act_train,act_test,people

In [112]:
train

,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,char_7_x,char_8_x,char_9_x,...,char_33,char_34,char_35,char_36,char_37,char_38,year_y,month_y,day_y,isweekend_y
0,4,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,36,2021,6,29,0
0,2,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,36,2021,6,29,0
0,2,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,36,2021,6,29,0
0,2,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,36,2021,6,29,0
0,2,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,36,2021,6,29,0
0,4,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,36,2021,6,29,0
1,2,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,76,2021,1,6,0
1,2,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,76,2021,1,6,0
2,2,0,0,0,0,0,0,0,0,0,...,1,1,0,1,1,99,2022,6,10,0
2,2,0,0,0,0,0,0,0,0,0,...,1,1,0,1,1,99,2022,6,10,0


In [129]:
colname = test.columns
category = ['activity_category', 'char_1_x', 'char_2_x', 'char_3_x', 'char_4_x',
       'char_5_x', 'char_6_x', 'char_7_x', 'char_8_x', 'char_9_x', 'char_10_x',
        'year_x', 'month_x', 'day_x', 'isweekend_x', 'char_1_y',
       'group_1', 'char_2_y', 'char_3_y', 'char_4_y', 'char_5_y', 'char_6_y',
       'char_7_y', 'char_8_y', 'char_9_y', 'year_y', 'month_y', 'day_y', 'isweekend_y']
X_train_total = train[category]
X_test_total = test[category]
y_train_total = train['outcome']
uncategory = []
for i in colname:
    if i not in category:
        uncategory.append(i)

ohencoder = OneHotEncoder()
X_train_total = ohencoder.fit_transform(X_train_total[category])
X_test_total = ohencoder.transform(X_test_total[category])

X_train_total = hstack((X_train_total,train[uncategory]))
X_test_total = hstack((X_test_total,test[uncategory]))

In [130]:
print(X_train_total.shape)
print(y_train_total.shape)
print(X_test_total.shape)

(2197291, 36845)
(2197291,)
(498687, 36845)


,char_10_y,char_11,char_12,char_13,char_14,char_15,char_16,char_17,char_18,char_19,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,1,0,0,1,1,0,1,0,0,0,...,0,1,1,0,0,1,1,1,0,36
0,1,0,0,1,1,0,1,0,0,0,...,0,1,1,0,0,1,1,1,0,36
0,1,0,0,1,1,0,1,0,0,0,...,0,1,1,0,0,1,1,1,0,36
0,1,0,0,1,1,0,1,0,0,0,...,0,1,1,0,0,1,1,1,0,36
0,1,0,0,1,1,0,1,0,0,0,...,0,1,1,0,0,1,1,1,0,36
0,1,0,0,1,1,0,1,0,0,0,...,0,1,1,0,0,1,1,1,0,36
1,0,0,1,1,0,0,0,1,0,0,...,0,1,1,1,1,1,1,1,0,76
1,0,0,1,1,0,0,0,1,0,0,...,0,1,1,1,1,1,1,1,0,76
2,1,1,1,1,1,1,0,1,0,1,...,0,0,1,1,1,1,0,1,1,99
2,1,1,1,1,1,1,0,1,0,1,...,0,0,1,1,1,1,0,1,1,99
